# Bubble and Quick sort

## Utility

You would to skip it first.

In [1]:
import random
import operator
import timeit
import statistics


idx = operator.itemgetter(0)
val = operator.itemgetter(1)


def pairs(arr):
    return zip(enumerate(arr), enumerate(arr[1:], 1))


def is_bad(pair):
    a, b = pair
    return val(a) > val(b)


def bad_pairs(arr):
    return filter(is_bad, pairs(arr))


def is_sorted(arr):
    return not any(bad_pairs(arr))


def swap(arr, a, b):
    arr[b], arr[a] = arr[a], arr[b]


ARRAY_LENGTH = 2000
    
    
def build_random_array():
    return [random.randint(0, 100) for _ in range(ARRAY_LENGTH)]
    

def print_array(arr):
    print(f'{arr[:10]} ... {arr[-10:]}')


def times_to_ms(times_sec):
    sec = statistics.mean(times_sec)
    ms = sec * 1000
    return round(ms)


def random_array_time_ms(sorting_function):
    return times_to_ms(timeit.repeat(f'{sorting_function.__name__}(arr)',
                                     setup='arr=build_random_array()',
                                     globals=globals(),
                                     number=1,
                                     repeat=3))


def sorted_array_time_ms(sorting_function, arr):
    return times_to_ms(timeit.repeat(f'sorting_function(arr)',
                                     globals=locals(),
                                     number=1,
                                     repeat=3))


def print_random_array_sorting(title, sorting_function):
    print(f'{title} - {random_array_time_ms(sorting_function)} ms')
    arr = build_random_array()
    print_array(arr)
    assert not is_sorted(arr)
    sorting_function(arr)
    assert is_sorted(arr)
    print_array(arr)
    print()


def print_sorted_array_sorting(title, sorting_function):
    arr = build_random_array()
    arr.sort()
    print(f'{title} - {sorted_array_time_ms(sorting_function, arr)} ms')
    sorting_function(arr)
    assert is_sorted(arr)
    print()


def print_single_value_array_sorting(title, sorting_function):
    arr = [0] * ARRAY_LENGTH
    assert is_sorted(arr)
    print(f'{title} - {sorted_array_time_ms(sorting_function, arr)} ms')
    sorting_function(arr)
    print()

## [Bubble sort](#bubble_sort)

In [2]:
def bubble_sort(arr):
    while not is_sorted(arr):
        for a, b in bad_pairs(arr):
            swap(arr, idx(a), idx(b))

## [Quick sort](#quick_sort)

### Partition scheme

All magic of sorting happens during partition process. We divide the array on two partitions: `[low .. pivot] <= [pivot .. high]`.

As a result we have the array, that sorted by **pivot**, and **pivot index**, that must be returned by `partition` function.

There are two most popular schemes of partition: Hoare and Lomuto. Lomuto's scheme is less efficient, but more simple.

#### Lomuto partition scheme

We choose the last array item `arr[high]` as **pivot**.

Then, we shifting low bound of array `low += 1` and compare the item `arr[low]` with **pivot**. If `pivot < item`, we place the item after the pivot.

In [3]:
def partition_lomuto(arr, low, high):
    while low < high:
        if arr[high] < arr[low]:
            arr.insert(high, arr.pop(low))
            high -= 1
        else:
            low += 1
    return low

#### Hoare partition scheme

We choose array middle item as **pivot**.

Then, we move `low` and `high` array bounds toward each other until they collide.

If we meet `pivot <= arr[low]` or `arr[high] <= pivot` values during moving array bounds, we swap it.

In [4]:
def partition_hoare(arr, low, high):
    pivot = arr[(low + high) // 2]
    low -= 1
    high += 1
    while True:
        low += 1
        while arr[low] < pivot:
            low += 1
        high -= 1
        while pivot < arr[high]:
            high -= 1
        if low >= high:
            return high + 1
        swap(arr, low, high)

### Main function

This is Quick sort entry point. We recursively repeating partition function.

In [5]:
# For ability to switch method
partition = partition_hoare


def quick_sort(arr, low=0, high=-1):
    high += 0 if 0 <= high else len(arr)
    if low < high:
        part = partition(arr, low, high)
        quick_sort(arr, low, part - 1)
        quick_sort(arr, part, high)

## [Testing](#testing)

In [6]:
print_random_array_sorting('Bubble sort, random array', bubble_sort)

partition = partition_lomuto
print_random_array_sorting('Quick sort Lomuto, random array', quick_sort)

partition = partition_hoare
print_random_array_sorting('Quick sort Hoare, random array', quick_sort)


print_sorted_array_sorting('Bubble sort, already sorted array', bubble_sort)

partition = partition_lomuto
print_sorted_array_sorting('Quick sort Lomuto, already sorted array', quick_sort)

partition = partition_hoare
print_sorted_array_sorting('Quick sort Hoare, already sorted array', quick_sort)


print_single_value_array_sorting('Bubble sort, array of one value', bubble_sort)

partition = partition_lomuto
print_single_value_array_sorting('Quick sort Lomuto, array of one value', quick_sort)

partition = partition_hoare
print_single_value_array_sorting('Quick sort Hoare, array of one value', quick_sort)

Bubble sort, random array - 3084 ms
[94, 24, 48, 16, 3, 79, 99, 10, 4, 55] ... [53, 54, 78, 62, 100, 31, 2, 28, 46, 48]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] ... [100, 100, 100, 100, 100, 100, 100, 100, 100, 100]

Quick sort Lomuto, random array - 24 ms
[68, 40, 87, 92, 1, 30, 41, 28, 10, 20] ... [24, 36, 28, 57, 50, 28, 32, 3, 4, 99]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] ... [100, 100, 100, 100, 100, 100, 100, 100, 100, 100]

Quick sort Hoare, random array - 14 ms
[49, 49, 38, 8, 4, 18, 68, 27, 53, 63] ... [90, 21, 82, 82, 45, 60, 39, 68, 91, 81]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] ... [100, 100, 100, 100, 100, 100, 100, 100, 100, 100]

Bubble sort, already sorted array - 1 ms

Quick sort Lomuto, already sorted array - 547 ms

Quick sort Hoare, already sorted array - 10 ms

Bubble sort, array of one value - 1 ms

Quick sort Lomuto, array of one value - 540 ms

Quick sort Hoare, array of one value - 14 ms

